In [105]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os
import copy
from re import sub
from decimal import Decimal
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


In [106]:
def curr_to_dec(money):
    return float(sub(r'[^\d.]', '', money))

In [107]:
def gdp_preproc(gdp_df):
    gdp_df = gdp_df[['GDP (nominal, 2017)','GDP per capita','Country']]
    gdp_df['GDP (nominal, 2017)'] = gdp_df['GDP (nominal, 2017)'].apply(curr_to_dec)
    gdp_df['GDP per capita'] = gdp_df['GDP per capita'].apply(curr_to_dec)
    return gdp_df
def preproc_col(df):
    return df.drop(columns=['Unnamed: 0','#']).rename(columns={"Country,Other":"Country"})
def merge_gdp(df,gdp_df=pd.read_csv("./data/gdp.csv")): 
    gdp_df = gdp_preproc(gdp_df)
    return gdp_df.merge(preproc_col(df),on='Country')
def add_death_per_cases(df):
    df['Deaths/Cases'] = df['TotalDeaths']/df['TotalCases']
    return df
def duy_preproc(df):
    result = copy.deepcopy(df)
    result = result.drop(df.tail(8).index)    #result = preproc_col(result)
    result = add_death_per_cases(result)
    result = merge_gdp(result)
    return result

In [108]:
df = pd.read_csv('./data/2022-04-27.csv')
df = duy_preproc(df)

/tmp/ipykernel_27335/2480772113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdp_df['GDP (nominal, 2017)'] = gdp_df['GDP (nominal, 2017)'].apply(curr_to_dec)
/tmp/ipykernel_27335/2480772113.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdp_df['GDP per capita'] = gdp_df['GDP per capita'].apply(curr_to_dec)


In [109]:
df.sample(5)

,"GDP (nominal, 2017)",GDP per capita,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop,Deaths/Cases
85,3.810773e+10,5791.0,Libya,501904,NaN,6430.0,NaN,490973.0,NaN,4501.0,101.0,71283.0,913.0,2477219.0,351830.0,7040960.0,Africa,14.0,1095.0,3.0,NaN,NaN,639.0,0.012811
32,3.147103e+11,10118.0,Malaysia,4440383,3471.0,35526.0,6.0,4346421.0,6900.0,58436.0,98.0,134073.0,1073.0,58544609.0,1767703.0,33119023.0,Asia,7.0,932.0,1.0,105.0,0.2,1764.0,0.008001
118,1.232286e+10,642.0,Burkina Faso,20853,NaN,382.0,NaN,20439.0,NaN,32.0,NaN,950.0,17.0,248995.0,11347.0,21943574.0,Africa,1052.0,57444.0,88.0,NaN,NaN,1.0,0.018319
50,1.397611e+11,14364.0,Hungary,1897897,2220.0,46162.0,29.0,1797144.0,4528.0,54591.0,38.0,197373.0,4801.0,11324260.0,1177671.0,9615808.0,Europe,5.0,208.0,1.0,231.0,3.0,5677.0,0.024323
110,1.501395e+10,7271.0,Gabon,47597,NaN,303.0,NaN,47282.0,NaN,12.0,NaN,20507.0,131.0,1592483.0,686100.0,2321065.0,Africa,49.0,7660.0,1.0,NaN,NaN,5.0,0.006366


In [110]:
df = df.dropna(axis=0,subset=['Deaths/1M pop','Tot\xa0Cases/1M pop'])

In [111]:
df_1 = df[["GDP per capita","Deaths/Cases"]]

In [116]:
f, ax = plt.subplots(figsize=(7, 7))
ax.set(xscale="log", yscale="log")
sns.regplot(x='GDP (nominal, 2017)', y='Deaths/Cases', data=df,ax = ax)


NameError: name 'ax' is not defined